<a href="https://colab.research.google.com/github/Romal27/DSGP-24-Retina94/blob/ChatBot-Development/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y nltk
!pip install nltk

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


In [2]:
import pandas as pd

# Loading the uploaded CSV file
intents_df = pd.read_csv('intents.csv')

# Displaying the first few rows to confirm it loaded correctly
print(intents_df.head())


          tag                                   patterns__001  \
0    greeting                                              Hi   
1       about                   What is diabetic retinopathy?   
2    symptoms  What are the symptoms of diabetic retinopathy?   
3      stages    What are the stages of diabetic retinopathy?   
4  prevention         How can I prevent diabetic retinopathy?   

                                   patterns__002  \
0                                          Hello   
1             Tell me about diabetic retinopathy   
2  How do I know if I have diabetic retinopathy?   
3     Explain the stages of diabetic retinopathy   
4               What are the prevention methods?   

                                     patterns__003 patterns__004  \
0                                              Hey  Good morning   
1                     Explain diabetic retinopathy           NaN   
2                 Symptoms of diabetic retinopathy           NaN   
3  How many stages does 

In [3]:
print(intents_df.columns)


Index(['tag', 'patterns__001', 'patterns__002', 'patterns__003',
       'patterns__004', 'patterns__005', 'responses__001', 'responses__002',
       'responses__003'],
      dtype='object')


We need to combine all the patterns__* columns into one column named pattern

In [4]:
pattern_columns = [col for col in intents_df.columns if col.startswith('patterns__')]
intents_df['pattern'] = intents_df[pattern_columns].apply(lambda row: ' '.join(row.dropna()), axis=1)


Similarly, combined all responses__* columns into one column named response

In [5]:
response_columns = [col for col in intents_df.columns if col.startswith('responses__')]
intents_df['response'] = intents_df[response_columns].apply(lambda row: ' '.join(row.dropna()), axis=1)


In [6]:
intents_df = intents_df[['tag', 'pattern', 'response']]
print(intents_df.head())


          tag                                            pattern  \
0    greeting             Hi Hello Hey Good morning Good evening   
1       about  What is diabetic retinopathy? Tell me about di...   
2    symptoms  What are the symptoms of diabetic retinopathy?...   
3      stages  What are the stages of diabetic retinopathy? E...   
4  prevention  How can I prevent diabetic retinopathy? What a...   

                                            response  
0  Hello! How can I assist you today? Hi there! H...  
1  Diabetic retinopathy is a complication of diab...  
2  Symptoms include blurred vision, difficulty se...  
3  The stages are Mild, Moderate, Severe, and Pro...  
4  Maintain blood sugar levels, eat healthy, and ...  


Tokenize and preprocess patterns

In [22]:
import nltk
nltk.data.path.clear()


In [23]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
# Checking for missing values in the 'pattern' column
print(intents_df['pattern'].isnull().sum())

# Display rows with missing 'pattern' values
print(intents_df[intents_df['pattern'].isnull()])


0
Empty DataFrame
Columns: [tag, pattern, response]
Index: []


In [29]:
import nltk
nltk.data.path.clear()  # Clear any existing NLTK cache
nltk.download('punkt')  # Re-download 'punkt'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
# Split concatenated patterns into individual rows
expanded_rows = []
for _, row in intents_df.iterrows():
    patterns = row['pattern'].split()  # Split patterns by whitespace
    for pattern in patterns:
        expanded_rows.append({'tag': row['tag'], 'pattern': pattern, 'response': row['response']})

# Create a new DataFrame with expanded rows
intents_df = pd.DataFrame(expanded_rows)

# Check the updated DataFrame
print(intents_df.head())


        tag  pattern                                           response
0  greeting       Hi  Hello! How can I assist you today? Hi there! H...
1  greeting    Hello  Hello! How can I assist you today? Hi there! H...
2  greeting      Hey  Hello! How can I assist you today? Hi there! H...
3  greeting     Good  Hello! How can I assist you today? Hi there! H...
4  greeting  morning  Hello! How can I assist you today? Hi there! H...


In [7]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk.tokenize import word_tokenize

# Test with a simple string
test_string = "Hello! How are you?"
tokens = word_tokenize(test_string)
print(f"Tokens: {tokens}")


Tokens: ['Hello', '!', 'How', 'are', 'you', '?']


In [6]:
print(intents_df.columns)


Index(['tag', 'patterns__001', 'patterns__002', 'patterns__003',
       'patterns__004', 'patterns__005', 'responses__001', 'responses__002',
       'responses__003'],
      dtype='object')


In [9]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

# Preprocess patterns
def preprocess_patterns(df):
    df['pattern_tokens'] = df['pattern'].apply(lambda x: [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(str(x))])
    return df

intents_df = preprocess_patterns(intents_df)

# Display the processed DataFrame
print(intents_df[['pattern', 'pattern_tokens']].head())


                                             pattern  \
0             Hi Hello Hey Good morning Good evening   
1  What is diabetic retinopathy? Tell me about di...   
2  What are the symptoms of diabetic retinopathy?...   
3  What are the stages of diabetic retinopathy? E...   
4  How can I prevent diabetic retinopathy? What a...   

                                      pattern_tokens  
0     [hi, hello, hey, good, morning, good, evening]  
1  [what, is, diabetic, retinopathy, ?, tell, me,...  
2  [what, are, the, symptom, of, diabetic, retino...  
3  [what, are, the, stage, of, diabetic, retinopa...  
4  [how, can, i, prevent, diabetic, retinopathy, ...  


In [10]:
intents_df.to_csv('intents_preprocessed.csv', index=False)
print("Preprocessed dataset saved successfully.")


Preprocessed dataset saved successfully.
